# LOAD DIGITS

In [99]:
from sklearn.datasets import load_digits

digits = load_digits()

In [100]:
digits.keys()

dict_keys(['data', 'target', 'frame', 'feature_names', 'target_names', 'images', 'DESCR'])

In [101]:
import pandas as pd
df = pd.DataFrame(digits['data']).astype(int)
df.columns = digits['feature_names']

In [102]:
df

,pixel_0_0,pixel_0_1,pixel_0_2,pixel_0_3,pixel_0_4,pixel_0_5,pixel_0_6,pixel_0_7,pixel_1_0,pixel_1_1,...,pixel_6_6,pixel_6_7,pixel_7_0,pixel_7_1,pixel_7_2,pixel_7_3,pixel_7_4,pixel_7_5,pixel_7_6,pixel_7_7
0,0,0,5,13,9,1,0,0,0,0,...,0,0,0,0,6,13,10,0,0,0
1,0,0,0,12,13,5,0,0,0,0,...,0,0,0,0,0,11,16,10,0,0
2,0,0,0,4,15,12,0,0,0,0,...,5,0,0,0,0,3,11,16,9,0
3,0,0,7,15,13,1,0,0,0,8,...,9,0,0,0,7,13,13,9,0,0
4,0,0,0,1,11,0,0,0,0,0,...,0,0,0,0,0,2,16,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1792,0,0,4,10,13,6,0,0,0,1,...,4,0,0,0,2,14,15,9,0,0
1793,0,0,6,16,13,11,1,0,0,0,...,1,0,0,0,6,16,14,6,0,0
1794,0,0,1,11,15,1,0,0,0,0,...,0,0,0,0,2,9,13,6,0,0
1795,0,0,2,10,7,0,0,0,0,0,...,2,0,0,0,5,12,16,12,0,0


In [103]:
df['target'] =  digits['target']
df.dropna(inplace=True)
target = df.pop('target')

In [104]:
import h5py
with h5py.File("usps.h5", 'r') as hf:
        train = hf.get('train')
        X_tr = train.get('data')[:]
        y_tr = train.get('target')[:]
        test = hf.get('test')
        X_te = test.get('data')[:]
        y_te = test.get('target')[:]

In [105]:
df = pd.DataFrame(X_tr)
target = y_tr

# Numerical UMAP

In [106]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import umap.umap_ as umap
from scipy.spatial.distance import cdist

In [107]:
numerical = df.select_dtypes('number')
categorical = df.select_dtypes('object')
scaler = StandardScaler()
numerical = scaler.fit_transform(numerical)
categorical = categorical.apply(lambda x: x.replace(x.unique(),list(range(1,1+len(x.unique())))))
gamma = np.mean(np.std(numerical))/2
distances = (np.nan_to_num(cdist(numerical,numerical,'sqeuclidean'))) + np.nan_to_num(cdist(categorical,categorical,'hamming'))*gamma
umap_embedding = umap.UMAP(n_components=len(df.columns),metric="precomputed").fit_transform(distances)

c:\Users\cleme\AppData\Local\Programs\Python\Python310\lib\site-packages\umap\umap_.py:1780: UserWarning: using precomputed metric; inverse_transform will be unavailable
  warn("using precomputed metric; inverse_transform will be unavailable")


In [109]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

raw_scores = cross_val_score(KNeighborsClassifier(15), df, target)

In [110]:
umap_scores = cross_val_score(KNeighborsClassifier(15), umap_embedding, target)

In [111]:
np.mean(raw_scores) , np.mean(umap_scores)

(0.9476053745213241, 0.9503488587462898)

In [112]:
df2 = df.copy()
df2.iloc[:,:128].apply(lambda x: pd.cut(x, bins = [0,128,256], labels=['low','high']))
numerical = df2.select_dtypes('number')
categorical = df2.select_dtypes('object')
scaler = StandardScaler()
numerical = scaler.fit_transform(numerical)
categorical = categorical.apply(lambda x: x.replace(x.unique(),list(range(1,1+len(x.unique())))))
gamma = np.mean(np.std(numerical))/2
distances = (np.nan_to_num(cdist(numerical,numerical,'sqeuclidean'))) + np.nan_to_num(cdist(categorical,categorical,'hamming'))*gamma
umap_embedding2 = umap.UMAP(n_components=len(df.columns),metric="precomputed").fit_transform(distances)

c:\Users\cleme\AppData\Local\Programs\Python\Python310\lib\site-packages\umap\umap_.py:1780: UserWarning: using precomputed metric; inverse_transform will be unavailable
  warn("using precomputed metric; inverse_transform will be unavailable")


In [113]:
umap_scores2 = cross_val_score(KNeighborsClassifier(15), umap_embedding2, target)

In [114]:
f"""Raw Coordinates  :{np.mean(raw_scores)} 
    Numerical UMAP : {np.mean(umap_scores)} 
    Mixed UMAP (32-32) :{np.mean(umap_scores2)}"""

'Raw Coordinates  :0.9476053745213241 \n    Numerical UMAP : 0.9503488587462898 \n    Mixed UMAP (32-32) :0.9496632697480564'